In [213]:
import numpy as np
import collections
import math
from tqdm import tqdm
from IPython.display import clear_output
import random
import copy
from nltk.tokenize import word_tokenize

In [2]:
def read_vectors():
    word2vecd = collections.defaultdict(lambda: None)
    with open("poleval_base_vectors.txt", "r") as f:
        data = f.read().split("\n")[:-1]
        for datarow in data:
            drowsplit = datarow.split()
            word = drowsplit[0]
            vector = [float(a) for a in drowsplit[1:]]
            word2vecd[word] = vector
    return word2vecd

In [3]:
word2vecd = read_vectors()

In [53]:
def worddistance(vec1, vec2):
    licznik = 0.0
    for i in range(len(vec1)):
        licznik += vec1[i] * vec2[i]

    m1 = math.sqrt(sum(a**2 for a in vec1))
    m2 = math.sqrt(sum(a**2 for a in vec2))
    return licznik/(m1 * m2)

In [56]:
word2vecd['kamień']

[0.140472,
 -0.158079,
 -0.044729,
 0.034694,
 0.013923,
 0.318778,
 -0.413295,
 0.151537,
 0.16049,
 -0.08871,
 -0.07402,
 0.132103,
 -0.094077,
 -0.058236,
 0.11688,
 0.182977,
 0.151061,
 0.051148,
 -0.537341,
 0.11795,
 -0.269385,
 0.017965,
 0.314765,
 0.447163,
 -0.335488,
 -0.147971,
 0.240297,
 -0.380222,
 0.113034,
 -0.304172,
 -0.027287,
 0.08845,
 -0.121732,
 0.2618,
 0.19022,
 -0.089543,
 0.03096,
 0.017614,
 -0.125878,
 0.267806,
 0.018364,
 -0.140993,
 -0.099959,
 -0.410951,
 0.084118,
 0.362173,
 -0.108261,
 0.246407,
 0.24926,
 -0.38413,
 0.187338,
 0.274851,
 -0.145103,
 -0.106627,
 -0.156686,
 -0.222085,
 0.451045,
 -0.208777,
 -0.179971,
 -0.257014,
 0.440965,
 -0.149437,
 0.059571,
 -0.094444,
 -0.174792,
 0.800863,
 0.079057,
 -0.029079,
 0.14488,
 0.256368,
 0.286435,
 0.46234,
 0.005087,
 0.082126,
 -0.199524,
 0.15393,
 -0.229041,
 -0.466491,
 -0.057133,
 -0.403586,
 0.069173,
 -0.122827,
 -0.064214,
 -0.160929,
 -0.208119,
 0.011429,
 0.637632,
 0.121419,
 -0.2

In [57]:
a = []
for word in tqdm(list(word2vecd.keys())):
    try:
        a.append((worddistance(word2vecd[word], word2vecd["kamień"]), word))
    except Exception as e:
        pass
    
for kit, k in list(enumerate(sorted(a, reverse=True)))[:10]:
    print(kit, k)

100%|██████████| 532224/532224 [00:13<00:00, 39383.47it/s]


0 (1.0, 'kamień')
1 (0.7056827799366562, 'głaz')
2 (0.7005540027232601, 'otoczak')
3 (0.6956038193214734, 'łupany')
4 (0.6724589621756244, 'żarnowe')
5 (0.6674419304660484, 'ciosany')
6 (0.6672049650767454, 'ociosany')
7 (0.6568904023407018, 'cegła')
8 (0.6556979853827196, 'półszlachetny')
9 (0.652511024703712, 'granitowy')


In [200]:
def succ_tags():
    word2base = collections.defaultdict(lambda: None)
    word2tag = collections.defaultdict(lambda: [])
    tag2words = collections.defaultdict(lambda: [])

    with open("skladnicaTagsBases.pl", "r") as f:
        for l in f.read().split("\n")[:-1]:
            try:
                tokd = l.split("(")[1].split(")")[0].replace(" ", "").split("'")
                word = tokd[1]
                base = tokd[3]
                tag = tokd[4][1:]
                word2base[word] = base
                word2tag[word].append(tag)
                #word2tag[base].append(tag)
                tag2words[tag].append(word)
                word2base[base] = base
            except Exception as e:
                print(l)
            
    return (word2base, word2tag, tag2words)

word2based, word2tagd, tag2wordsd = succ_tags()

tagAndBase('(', '(', interp).
tagAndBase(')', ')', interp).
tagAndBase('ry(d)zykować', 'ryzykować', inf:imperf).


In [201]:
def succ_potencjalne_rymy():
    rhimes = []
    with open("rytmiczne_zdania_z_korpusu.txt", "r") as f:
        for line in f.read().split("\n")[:-1]:
            linel, linep = line.split("[*]")
            linel = " ".join(linel.split(" ")[1:])
            rhimes.append((word_tokenize(linel), word_tokenize(linep)))
    return rhimes
            
rhimes = succ_potencjalne_rymy()
print(rhimes[0])

(['aby', 'je', 'zwalczyć', 'często', 'stosuje', 'się', 'silne'], ['antybiotyki', ',', 'ale', 'nieraz', 'nieskuteczne', '.'])


In [256]:
def count_sylaby(w):
    for l in 'aeioóuyąę':
        w = w.replace(f"i{l}", '#')

    return max(sum([1 for litera in w if litera in 'aeioóuyąę#']), 0)

def getrhime(word):
    target = count_sylaby(word) - 1
    for i in range(1, len(word)+1):
        if count_sylaby(word[:i]) == target:
            return word[i-1:]
    return word

def getrhimespace(corpus):
    rhimespace = set()
    for word in corpus:
        rhimespace.add(getrhime(word))
    return rhimespace

rhimespace = getrhimespace(list(word2based.keys()))

def characterize_target(sentence, verbose):
    lastword = ""
    wpos = len(sentence)-1
    for tok in sentence[::-1]:
        if len(tok) > 1:
            if verbose:
                print(tok)
            lastword = tok
            break
        wpos -= 1
    
    tag = word2tagd[lastword]
    tagp = word2tagd[sentence[wpos-1]]
    base = word2based[lastword]
    if verbose:
        print(f"lastword is {lastword}")
        print(f"tag is {tagp} -> {tag}")
        print(f"base is {base}")
    wvec = word2vecd[lastword]
    wbase = word2vecd[word2based[lastword]]
    syllabec = count_sylaby(lastword)
    rhimefactor = getrhime(lastword)
    return(lastword, tag, base, wvec, wbase, syllabec, rhimefactor, wpos)

def get_r2wd(corpus):
    rhime2word = collections.defaultdict(lambda: [])
    for word in corpus:
        rhime2word[getrhime(word)].append(word)
    return rhime2word

rhime2wordd = get_r2wd(list(word2tagd.keys()))

def get_candidates(tag, rhime, sylablecount):
    candidates = []
    for word in rhime2wordd[rhime]:
        if word:
            if count_sylaby(word) == sylablecount:
                if any(x in tag for x in word2tagd[word]):
                    if word.endswith(rhime):
                        candidates.append(word)
    return candidates

def get_candidates_for_rhimebases(rhimebase, awvec, awbase, atag, asc, bwvec, bwbase, btag, bsc, aword, bword, verbose):
    final_candidate_space = []
    for rhime in rhimebase:
        if verbose:
            print(f"try {rhime}")
        acandidates = []
        bcandidates = []

        if awvec or awbase:
            acandidates = get_candidates(atag, rhime, asc)
        else:
            acandidates = [aword]

        if bwvec or bwbase:
            bcandidates = get_candidates(btag, rhime, bsc)
        else:
            bcandidates = [bword]

        if verbose:
            print(f"continue with AC = {len(acandidates)}")
            print(f"continue with BC = {len(bcandidates)}")
        if len(acandidates) * len(bcandidates) > 0:
            final_candidate_space.append([acandidates, bcandidates])
    return final_candidate_space

def render_candidates(groupa, fav):
    acanrender = []
    for candidate in groupa:
        try:
            acanrender.append((worddistance(fav, word2vecd[candidate]), candidate))
        except Exception:
            try:
                acanrender.append((worddistance(fav, word2vecd[word2based[candidate]]), candidate))
            except Exception:
                acanrender.append((0.00001, candidate))
                pass
            pass
        
    return acanrender


def rythmize_easy(target_line, verbose = False, outcount = 2):
    linef, lines = target_line
    print(linef)
    print(lines)
    aword, atag, abase, awvec, awbase, asc, arhime, awpos = characterize_target(linef, verbose)
    bword, btag, bbase, bwvec, bwbase, bsc, brhime, bwpos = characterize_target(lines, verbose)
    if (awvec or awbase) or (bwvec or bwbase):
        # make mind over base.
        rhimebase = ""
        if not (awvec or awbase):
            rhimebase = [arhime]
            if verbose:
                print("no a vector!")
        elif not (bwvec or bwbase):
            rhimebase = [brhime]
            if verbose:
                print("no b vector!")
        else:
            rhimebase = list(set([arhime, brhime]))
        if verbose:
            print(f"continue with rhimebase {rhimebase}")
        
        # find suitable targets.
        productions = get_candidates_for_rhimebases(rhimebase, awvec, awbase, atag, asc, bwvec, bwbase, btag, bsc, aword, bword, verbose)

        for rb in rhimespace:
            acandidates = get_candidates(atag, rb, asc)    
            bcandidates = get_candidates(btag, rb, bsc)  

            if len(acandidates) * len(bcandidates) > 0:
                productions.append([acandidates, bcandidates])   
                if verbose:
                    print(f"try {rb} -> {acandidates} {bcandidates}")

        fav = awvec if awvec else awbase
        fbv = bwvec if bwvec else bwbase
        
        rhimerender = []
        if verbose:
            print("=======")
            print(f"{len(productions)} candidates for rhimes!")

        for candidate_group in productions:
            if verbose:
                print(candidate_group)
            groupa, groupb = candidate_group
            acanrender = render_candidates(groupa, fav)
            bcanrender = render_candidates(groupb, fbv)
            if verbose:
                print(acanrender, bcanrender)
            if len(bcanrender) * len(acanrender) > 0:
                if verbose:
                    print("the rhimegroup has useful productions!")
                for ac in list(sorted(acanrender)):
                    for bc in list(sorted(bcanrender)):
                        rhimerender.append((ac[0]*bc[0], ac, bc))
        
        if len(rhimerender) == 0:
            raise NotImplemented
        
        emitted = 0
        for prod in list(sorted(list(set(rhimerender)), reverse=True)):
            prodscore, pa, pb = prod
            if pa[1] != aword and pb[1] != bword:
                print(f"with confidence {prodscore}")
                bakef = copy.deepcopy(linef)
                bakes = copy.deepcopy(lines)
                bakef[int(awpos)] = pa[1]
                bakes[int(bwpos)] = pb[1]
                print(" ".join(bakef))
                print(" ".join(bakes))
                print()
                emitted += 1
                if emitted == outcount:
                    return
    else:
        print("No vectors found, terminating")
    

rythmize_easy(copy.deepcopy(rhimes[13]))

['autentyczna', 'godność', 'i', 'wolność', 'człowieka']
['wymaga', 'zachowania', 'prawa', 'moralnego', '.']


In [257]:
f = 0
for t in rhimes:
    try:
        print("======================")
        rythmize_easy(copy.deepcopy(t))
    except Exception:
        f += 1
        pass
    
print(f"failed {f} of {len(rhimes)}")

['aby', 'je', 'zwalczyć', 'często', 'stosuje', 'się', 'silne']
['antybiotyki', ',', 'ale', 'nieraz', 'nieskuteczne', '.']
['a', 'bywa', 'tak', 'dlatego', ',', 'że', 'wobec', 'tych', ',', 'którzy']
['nam', 'się', 'nie', 'podobają', 'bywamy', 'zołzami', '.']
['a', 'do', 'powstania', 'takiej', 'sytuacji', 'naród']
['nie', 'upoważnił', 'swoich', 'elit', 'politycznych', '.']
['a', 'jakiej', 'wysokości', 'jest', 'kontrakt', ',', 'zależy']
['od', 'tego', ',', 'ile', 'mamy', 'pieniędzy', 'w', 'systemie', '.']
['ale', 'jeśli', 'pan', 'zrobi', 'sondaż', 'w', 'społeczeństwie']
[',', 'będzie', 'to', 'wyglądało', 'zupełnie', 'inaczej', '.']
['ale', ',', 'panie', 'marszałku', ',', 'kwestia', 'ta', 'podlega']
['rozstrzygnięciu', 'przez', 'pana', 'lub', 'prezydium', 'sejmu', '.']
['alfons', 'na', 'ogół', 'bardzo', 'surowy', 'w', 'ściganiu']
['przestępstw', ',', 'nie', 'podjął', 'żadnych', 'kroków', 'w', 'tym', 'kierunku', '.']
['angielska', 'ariergarda', ',', 'strzelając', 'z', 'zawietrznej']
[',', 'm

['buntownicy', 'włamali', 'się', 'do', 'magazynów']
[',', 'popili', 'się', ',', 'a', 'większość', 'zapasów', 'zniszczyli', '.']
No vectors found, terminating
['burmistrz', 'miasta', 'lublińca', 'oraz', 'środowiska']
['harcerskie', 'i', 'zuchowe', 'z', 'terenu', 'powiatu', '.']
with confidence 0.20467108868418718
burmistrz miasta lublińca oraz zachowania
harcerskie i zuchowe z terenu poznania .

with confidence 0.20312860509868916
burmistrz miasta lublińca oraz współdziałania
harcerskie i zuchowe z terenu poznania .

['byłabym', 'wdzięczna', 'panu', 'posłowi', 'z', 'unii']
['wolności', ',', 'gdyby', 'wskazał', ',', 'z', 'czego', 'to', 'wynika', '.']
with confidence 0.12036298715973329
byłabym wdzięczna panu posłowi z osi
wolności , gdyby wskazał , z czego to donosi .

with confidence 0.11244993491879371
byłabym wdzięczna panu posłowi z osi
wolności , gdyby wskazał , z czego to odnosi .

['była', 'muzyka', 'grana', 'na', 'żywo', 'przez', 'pana']
['marka', 'szerszenia', ',', 'były', 'zaba

['cristina', 'wciąż', 'unika', 'swojego', 'chłopaka']
[',', 'po', 'tym', 'jak', 'dowiedziała', 'się', ',', 'że', 'ma', 'problemy', '.']
with confidence 0.1463419978209079
cristina wciąż unika swojego osamy
, po tym jak dowiedziała się , że ma programy .

with confidence 0.1321151551505963
cristina wciąż unika swojego poety
, po tym jak dowiedziała się , że ma konkrety .

['cukinię', 'i', 'paprykę', 'pokrój', 'w', 'drobne', 'paski']
[',', 'aby', 'przypominały', 'makaron', 'spaghetti', '.']
['czas', 'pokaże', ',', 'na', 'ile', 'te', 'optymistyczne']
['zapowiedzi', 'zostaną', 'zrealizowane', '.']
['część', 'już', 'się', 'nauczyła', 'i', 'omija', 'pana']
['pilarskiego', 'i', 'jego', 'przekręty', 'z', 'daleka', '!']
with confidence 0.1489761243167512
część już się nauczyła i omija jana
pilarskiego i jego przekręty z udana !

with confidence 0.10044279477781791
część już się nauczyła i omija tuska
pilarskiego i jego przekręty z francuska !

['często', 'utożsamiany', 'z', 'prostym', 'urządzen

['dodać', 'przesianą', 'mąkę', 'z', 'proszkiem', 'do', 'pieczenia']
['i', 'wymieszać', 'aż', 'masa', 'będzie', 'jednolita', '.']
No vectors found, terminating
['dodać', 'przy', 'tym', 'należy', ',', 'że', 'klasyfikacja']
['statku', 'jest', 'dobrowolna', 'w', 'myśl', 'przepisów', 'prawa', '.']
['dodatkowe', 'oceny', 'uczeń', 'otrzymuje']
['za', 'niektóre', 'z', 'pisanych', 'w', 'domu', 'wypracowań', '.']
['dodatkowo', 'powstają', 'nowe', 'technologie']
[',', 'nowe', 'lepsze', 'i', 'droższe', 'metody', 'leczenia', '.']
['dojazd', 'do', 'działki', 'drogą', 'asfaltową', ',', 'gminną']
[',', 'ostatnie', 'parę', 'metrów', ',', 'drogą', 'utwardzoną', '.']
No vectors found, terminating
['do', 'kradzieży', 'ciągnika', 'z', 'przyczepą', 'z', 'terenu']
['niemiec', ',', 'doszło', 'jesienią', 'ubiegłego', 'roku', '.']
with confidence 6.9037231593279164e-06
do kradzieży ciągnika z przyczepą z obszaru
niemiec , doszło jesienią ubiegłego czaru .

with confidence 6.9037231593279164e-06
do kradzieży cią

['dzięki', 'temu', 'możemy', 'jej', 'używać', 'nawet']
['wtedy', ',', 'gdy', 'mamy', 'mokre', 'lub', 'wilgotne', 'dłonie', '.']
['dzięki', 'temu', 'zdobędzie', 'popularność', ',', 'rozgłos']
['i', 'wielu', 'klientów', ',', 'jacy', 'przysporzą', 'jej', 'zysków', '.']
['dziękuję', 'bardzo', 'panu', 'posłowi', 'i', 'bardzo']
['dziękuję', 'za', 'zmieszczenie', 'się', 'w', 'limicie', 'czasu', '.']
['dziewczynek', 'akcesoria', 'dla', 'lalek', 'rowerki']
['i', 'pojazdy', 'dla', 'dzieci', 'sanki', 'i', 'ślizgacze', '.']
No vectors found, terminating
['dzisiaj', 'istnieje', 'taki', 'mechanizm', ',', 'że', 'rada']
['ministrów', 'podejmuje', 'podobne', 'decyzje', '.']
['dzisiaj', 'w', 'wielu', 'tysiącach', 'wsi', 'polskich', 'brakuje']
['wody', ',', 'a', 'w', 'wielu', 'setkach', 'nie', 'ma', 'jej', 'w', 'ogóle', '.']
['eksperci', 'sfery', 'ducha', 'uznali', ',', 'że', 'główne']
['skrzyżowanie', 'w', 'miasteczku', 'będzie', 'idealne', '.']
['elektroniczny', 'dziennik', 'współgra', 'z', 'tradycyjny

...

['wskaźnik', 'ten', 'informuje', 'o', 'udziale', 'kosztów']
['ogólnozakładowych', 'w', 'wartości', 'sprzedaży', '.']
with confidence 1.863253762111393e-06
wskaźnik ten informuje o udziale bali
ogólnozakładowych w wartości centrali .

with confidence 1.0000000000000002e-10
wskaźnik ten informuje o udziale bali
ogólnozakładowych w wartości oddali .

['w', 'sklepie', 'zaopatrywać', 'się', 'mogą', 'podmioty']
['gospodarcze', ',', 'a', 'także', 'osoby', 'prywatne', '.']
['w', 'śląskiej', 'spółce', 'cukrowej', 'prezesi', 'robili']
['wszystko', ',', 'żeby', 'ogłaszać', 'upadłość', 'cukrowni', '.']
No vectors found, terminating
['wstępne', 'prognozy', 'mówią', 'o', 'dwutygodniowej']
['pauzie', 'zawodnika', 'unii', 'hrubieszów', '.']
['w', 'sumie', 'to', 'trzymam', 'kciuki', 'żeby', 'wszystkie', 'nasze']
['dzieciątka', 'urodziły', 'się', 'całe', 'i', 'zdrowe', '.']
['w', 'swej', 'istocie', 'zawiera', 'on', 'pewną', 'stabilność']
[',', 'która', 'jednak', 'nie', 'może', 'oznaczać', 'bezwładu', '.

with confidence 0.2732961514371895
w zawodach pucharowych sześć razy wygrała
na podium , przy czym tylko raz nie rozegrała .

with confidence 0.26638538760437863
w zawodach pucharowych sześć razy rozbiła
na podium , przy czym tylko raz nie wyprzedziła .

['w', 'zawodach', 'wzięli', 'udział', 'nie', 'tylko', 'polacy']
[',', 'ale', 'także', 'rosjanie', 'i', 'białorusini', '.']
['w', 'związku', 'z', 'tym', 'jest', 'mi', 'bardzo', 'trudno', 'w', 'tym', 'momencie']
['cokolwiek', 'komentować', ',', 'jest', 'to', 'wola', 'sejmu', '.']
with confidence 0.10348181603922499
w związku z tym jest mi bardzo trudno w tym wyniku
cokolwiek komentować , jest to wola wiku .

with confidence 0.10151569629411467
w związku z tym jest mi bardzo trudno w tym rozkroku
cokolwiek komentować , jest to wola bloku .

['w', 'zwołanej', 'z', 'tej', 'okazji', 'sesji', 'rady', 'miasta']
['kościerzyna', 'wziął', 'udział', 'poseł', 'zdzisław', 'czucha', '.']
['zabezpiecz', 'stolca', 'swego', 'lub', 'narzeczonego']
['i', 

with confidence 0.1833052926889478
z naklejki dołączonej do płyty kucharzy
, że zrobiono to w celu zmylenia handlarzy .

with confidence 0.1442067111474017
z naklejki dołączonej do płyty tłumaczy
, że zrobiono to w celu zmylenia działaczy .

['znalazł', 'się', 'wśród', 'najczęściej', 'cytowanych', 'polskich']
['naukowców', 'z', 'dziedziny', 'nauk', 'biomedycznych', '.']
['znaleziono', 'też', 'srebrny', 'denar', 'faustyny']
['starszej', ',', 'żony', 'cesarza', 'antonina', 'piusa', '.']
['z', 'naszym', 'bonem', 'otrzymasz', 'go', 'w', 'rewelacyjnej']
['cenie', ',', 'co', 'jeszcze', 'bardziej', 'poprawi', 'ci', 'humor', '.']
['zobacz', 'inne', 'papiery', 'i', 'tarcze', 'szlifierskie']
[',', 'najtańsze', 'i', 'najlepsze', 'oferty', ',', 'opinie', '.']
['zobowiązania', 'pewne', 'podjęła', 'wysoka']
['rada', ',', 'a', 'teraz', 'chcemy', 'obniżać', 'podatki', '.']
['została', 'pochowana', 'na', 'nowym', 'cmentarzu']
['żydowskim', 'przy', 'ulicy', 'miodowej', 'w', 'krakowie', '.']
with confide

['jak', 'była', 'na', 'początku', ',', 'tak', 'zawsze', 'niech', 'będzie']
[',', 'teraz', 'i', 'na', 'wiek', 'wieków', 'niechaj', 'słynie', 'wszędzie', '.']
['jesteś', 'bielsza', 'od', 'soli', 'i', 'bielsza', 'od', 'kredy']
[',', 'gdy', 'tak', 'za', 'mnie', 'spożywasz', 'gorycze', 'i', 'biedy', '.']
['jest', 'też', 'zintegrowane', 'z', 'systemem', 'księgowym']
[',', 'a', 'karty', 'pracy', 'również', 'z', 'systemem', 'płacowym', '.']
No vectors found, terminating
['jest', 'to', 'parów', 'tak', 'dziki', ',', 'że', 'nawet', 'roślinie']
['nie', 'da', 'kwiatów', 'rozwinąć', 'w', 'swej', 'pustej', 'krainie', '.']
No vectors found, terminating
['jest', 'to', 'poprawka', ',', 'która', 'dotyczy', 'słynnego']
['bankowego', 'tytułu', 'egzekucyjnego', '.']
['każda', 'firma', 'dodaje', 'do', 'wyposażenia']
['kuchni', 'co', 'najmniej', 'jedną', 'blachę', 'do', 'pieczenia', '.']
No vectors found, terminating
['kolor', 'musiał', 'pochodzić', 'od', 'słońca', 'promieni']
['którymi', 'przy', 'zachodzie',

['taki', 'układ', 'zapewniał', 'działanie', 'całego']
['systemu', 'również', 'podczas', 'lotu', 'nurkowego', '.']
['tam', 'gdzie', 'kiedyś', 'potrzebne', 'było', 'pozwolenie']
['na', 'budowę', ',', 'obecnie', 'wystarczy', 'zgłoszenie', '.']
['tam', ',', 'gdzie', 'kiedyś', 'potrzebne', 'było', 'zezwolenie']
['na', 'budowę', ',', 'obecnie', 'wystarczy', 'zgłoszenie', '.']
['te', 'drogi', 'w', 'dużej', 'części', 'są', 'takiej', 'jakości']
[',', 'która', ',', 'łagodnie', 'mówiąc', ',', 'budzi', 'wątpliwości', '.']
['teraz', 'nie', 'wiem', 'czy', 'moda', 'i', 'nas', 'starych', 'zmienia']
[',', 'czy', 'młodzież', 'lepsza', ',', 'ale', 'widzę', 'mniej', 'zgorszenia', '.']
['to', 'dwa', 'boże', 'kulawce', ',', 'dwa', 'rzewne', 'cudaki']
['kuleją', 'byle', 'jako', 'w', 'świat', 'nie', 'byle', 'jaki', '!']
['trudno', 'ukryć', 'wielbłąda', ',', 'byka', 'trudna', 'rada']
[',', 'lecz', 'jakże', 'łatwo', 'ukryć', 'małego', 'owada', '!']
['trzepocące', 'gromady', 'wron', 'złowróżbnie', 'kraczą']
[','